# Triaxial $\theta_E$ VS spherical $\theta_E$ for a selected sample from the TNG100 simulation

Data: a selected triaxial sample from the TNG100 simulation.

In [1]:
import pandas as pd 
import numpy as np 
from copy import deepcopy

In [2]:
cat100 = pd.read_pickle('../tng_catalog/tng_sigma_shape_cut/tng_100_finite_sigma_shape_cut.pkl')
cat100

,sigma_random_los,Re,halo_id,zeta,xi,triaxiality
0,198.717830,20.744231,1,0.816780,0.557836,0.483248
1,154.372133,8.852646,5,0.911836,0.531326,0.234857
2,202.764653,8.902069,6,0.870733,0.533219,0.337894
3,173.246111,6.141364,7,0.686777,0.606838,0.836311
5,160.809888,8.653671,9,0.853668,0.570757,0.402309
...,...,...,...,...,...,...
2048,151.884294,4.356955,60743,0.604044,0.468143,0.813391
2057,179.240207,11.454192,233214,0.921951,0.485322,0.196225
2059,170.711043,11.915249,255614,0.885115,0.326100,0.242342
2060,206.136011,13.277669,265267,0.864603,0.537626,0.355100


In [3]:
from lenstronomy.Analysis.lens_profile import LensProfileAnalysis
from deproject.projection import Projection
from deproject.Profiles.SIS_truncated_angular import SIS_truncated_angular
from deproject.Util.orientation import Sphere_random_point
from deproject.Cosmo.default_cosmo import get_default_lens_cosmo

In [4]:
# separate the prolate and oblate sample
oblate_mask = np.where(cat100['triaxiality'] <= 0.5, True, False)
cat_oblate = cat100.iloc[oblate_mask]
cat_prolate = cat100.iloc[~oblate_mask] 

In [5]:
def tng100_rmproj(catalog):
    """project a sample of galaxies each for several times and calculate the projected observables (Einstein radius, projected ellipticity, and orientation angle)

    Args:
        catalog (_type_): _description_

    Returns:
        _type_: _description_
    """
    # define a cosmology
    lens_cosmo = get_default_lens_cosmo()
    # calculate sigma_crit for kappa profile 
    sigma_crit = lens_cosmo.Sigma_crit / 1e6 # [M_sun/kpc^2]

    # access catalog data
    num_halo = len(catalog)
    sigma_rm_all = catalog['sigma_random_los'].values
    zeta_all = catalog['zeta'].values
    xi_all = catalog['xi'].values

    # thetaE_rm = lens_cosmo.sis_sigma_v2theta_E(sigma_rm_all)

    # each halo project 4 times
    times_single_proj = 4
    total_iteration = times_single_proj * num_halo

    # generate random LoS
    theta_all, phi_all = Sphere_random_point(num = total_iteration)

    # initialize some empty arrays for lensing observables
    thetaE_all = np.zeros(total_iteration)
    ellipticity_all = np.zeros(total_iteration)
    # sigma_sis_all = np.zeros(total_iteration)

    # define radius variables for integration and interpolation
    R_int = np.logspace(np.log10(0.0001), np.log10(150), num = 100)
    R_interp = np.logspace(np.log10(0.001), np.log10(50), num = 200)

    for i, sigma_rm in enumerate(sigma_rm_all):

        profile_sis = SIS_truncated_angular(sigma_v = sigma_rm, lens_cosmo = lens_cosmo)

        zeta = zeta_all[i]
        xi = xi_all[i]

        for j in range(times_single_proj):

            theta = theta_all[i * times_single_proj + j]
            phi = phi_all[i * times_single_proj + j]

            projection = Projection(zeta=zeta, xi=xi, theta=theta, phi=phi)

            radial_sis = projection.RadialProfile(R=R_int, profile=profile_sis, R_align='average', interpolate=True)
            radial_sis = radial_sis(R_interp)

            ellipticity_all[i * times_single_proj + j] = projection.Ellipticity()

            kappa = radial_sis / sigma_crit
            thetaE_all[i * times_single_proj + j] = LensProfileAnalysis.effective_einstein_radius_from_radial_profile(R_interp, kappa)

    # compute sigma_sis with einstein radius
    # sigma_sis_all = lens_cosmo.sis_theta_E2sigma_v(thetaE_all)

    # save data
    data_all = np.vstack([np.repeat(sigma_rm_all, times_single_proj), thetaE_all, ellipticity_all, theta_all, phi_all])

    return data_all


In [6]:
proj_oblate = False
if proj_oblate:
    data_oblate = tng100_rmproj(cat_oblate)
    np.save('./data_tng100_rmproj_oblate.npy', data_oblate)
else:
    data_oblate = np.load('./data_tng100_rmproj_oblate.npy')

In [7]:
proj_prolate = False
if proj_prolate:
    data_prolate = tng100_rmproj(cat_prolate)
    np.save('./data_tng100_rmproj_prolate.npy', data_prolate)
else:
    data_prolate = np.load('./data_tng100_rmproj_prolate.npy')